In [1]:
import sys
sys.path.append("../")
import snappy_utils
import os
from datetime import date, datetime
import settings
import zipfile

In [2]:
processed_data_path = settings.processed_data_path
raw_data_path = settings.raw_data_path

In [3]:
dates_list = ['2015-12-30', '2016-01-07', '2016-01-13', '2016-01-23', '2016-01-27',
 '2016-02-03', '2016-02-10', '2016-02-18', '2016-02-24', '2016-03-02',
 '2016-03-10', '2016-03-22', '2016-03-29', '2016-04-07', '2016-12-01',
 '2016-12-09', '2016-12-16', '2016-12-22', '2017-01-02', '2017-01-11',
 '2017-01-18', '2017-01-24', '2017-02-02', '2017-02-08', '2017-02-15',
 '2017-02-22', '2017-03-01', '2017-03-08', '2017-03-16', '2017-03-23',
 '2017-03-31', '2017-04-07', '2017-11-22', '2017-11-30', '2017-12-07',
 '2017-12-15', '2017-12-21', '2017-12-28', '2018-01-03', '2018-01-11',
 '2018-01-18', '2018-01-25', '2018-01-31', '2018-02-08', '2018-02-15',
 '2018-02-22', '2018-03-01', '2018-03-07', '2018-03-26', '2018-04-05',
 '2018-04-18', '2018-11-13', '2018-11-20', '2018-11-30', '2018-12-13',
 '2018-12-20', '2018-12-28', '2019-01-04', '2019-01-11', '2019-01-18',
 '2019-01-24', '2019-01-31', '2019-02-07', '2019-02-15', '2019-02-21',
 '2019-03-01', '2019-03-07', '2019-03-14', '2019-03-22', '2019-03-29',
 '2019-04-04', '2019-04-12', '2019-05-18', '2019-08-01', '2019-09-22',
 '2019-11-06', '2019-11-15', '2019-11-27', '2019-12-13', '2019-12-23',
 '2019-12-30', '2020-01-07', '2020-01-16', '2020-01-24', '2020-01-31',
 '2020-02-06', '2020-02-11', '2020-02-19', '2020-02-27', '2020-03-04',
 '2020-03-20', '2020-03-26', '2020-04-04', '2020-04-11', '2020-04-18',
 '2020-04-23', '2020-05-02', '2020-05-10']

downloaded_OLCI_paths = {}
for d in dates_list:
    downloaded_OLCI_paths[d] = []
    try:
        raw_data_dirs = [os.path.join(raw_data_path, d, s) for s in os.listdir(os.path.join(raw_data_path, d))]
    except FileNotFoundError:
        continue
    
    for f in raw_data_dirs:
        if "S3" in f and "OL" in f:
            downloaded_OLCI_paths[d].append(f)
        
dates_list = [datetime.strptime(d, "%Y-%m-%d").date() for d in dates_list]

### Make subsets of downloaded products

In [8]:
python_exec_path = sys.executable
olci_processing_script_dir = "/home/ubuntu/cianobacterias/src/"
olci_processing_script_path = os.path.join(olci_processing_script_dir, "OLCI_processing.py")

for d, paths in downloaded_OLCI_paths.items():
    if len(paths) == 0:
        continue
    for p in paths:
        input_data_path = p
        output_data_path = os.path.join(processed_data_path, d, "OLCI", "subselection")
        
        run_command = f"cd {olci_processing_script_dir} ; {python_exec_path} {olci_processing_script_path} -i {input_data_path} -o {output_data_path}"
        
        print(os.popen(run_command).read())

/home/ubuntu/cianobacterias/data/processed/2016-12-22/OLCI/subselection.dim already created

/home/ubuntu/cianobacterias/data/processed/2017-01-02/OLCI/subselection.dim already created


100% done.
Writing BEAM-DIMAP to /home/ubuntu/cianobacterias/data/processed/2017-01-11/OLCI/subselection.dim
Writing GeoTiff to /home/ubuntu/cianobacterias/data/processed/2017-01-11/OLCI/subselection.tif


100% done.
Writing BEAM-DIMAP to /home/ubuntu/cianobacterias/data/processed/2017-01-18/OLCI/subselection.dim
Writing GeoTiff to /home/ubuntu/cianobacterias/data/processed/2017-01-18/OLCI/subselection.tif


100% done.
Writing BEAM-DIMAP to /home/ubuntu/cianobacterias/data/processed/2017-02-02/OLCI/subselection.dim
Writing GeoTiff to /home/ubuntu/cianobacterias/data/processed/2017-02-02/OLCI/subselection.tif


100% done.
Writing BEAM-DIMAP to /home/ubuntu/cianobacterias/data/processed/2017-02-15/OLCI/subselection.dim
Writing GeoTiff to /home/ubuntu/cianobacterias/data/processed/2017-02-15/OLCI/subselec

### Reprocess downloaded data to have BEAM-DIMAP with pixel GeoCoding

In [ ]:
# iterate over the directorys containing every day's data
for directory in os.listdir(DATA_PATH):
    dir = os.path.join(DATA_PATH, directory)
    print(dir)
    index = 0
    # iterate over the content of those directorys
    for d in os.listdir(dir):
        # find the S3 downloaded data and make subset
        directory_content = os.path.join(DATA_PATH, directory, d)
        if os.path.isdir(directory_content) and ("S3" in d):
            index += 1
            dim_name = os.path.join(DATA_PATH, directory, "laguna_PixelGeoCoding") + str(index) + ".dim"
            print(dim_name)
            print(os.path.isfile(dim_name))
            #If BEAM-DIMAP is already generated, go to next day directory
            if (os.path.isfile(dim_name)):
                continue
            product = snappy_utils.read_product(directory_content)
            subset = snappy_utils.make_subset(product, settings.footprint)
            snappy_utils.write_product(subset, os.path.join(DATA_PATH, directory, "laguna_PixelGeoCoding") + str(index) + ".dim",
                                       "BEAM-DIMAP")

In [ ]:
# iterate over the directorys containing every day's data
for directory in os.listdir(DATA_PATH):
    dir = os.path.join(DATA_PATH, directory)
    print(dir)
    index = 0
    # iterate over the content of those directorys
    for d in os.listdir(dir):
        # find the S3 downloaded data and make subset
        directory_content = os.path.join(DATA_PATH, directory, d)
        if os.path.isdir(directory_content) and ("S3" in d):
            print(os.path.join(DATA_PATH, dir, d)